In [21]:
import feedparser
from docx import Document
from datetime import datetime

def search_arxiv_to_word_sorted(query, max_results):
    filename=query + ".docx"
    query = "+".join(query.split())
    base_url = 'http://export.arxiv.org/api/query?'
    query_url = f'search_query=all:{query}&start=0&max_results={max_results}'
    url = base_url + query_url
    
    feed = feedparser.parse(url)

    # Sort entries by published date (descending)
    entries = sorted(
        feed.entries,
        key=lambda entry: datetime.strptime(entry.published, '%Y-%m-%dT%H:%M:%SZ'),
        reverse=True
    )

    # Create Word doc
    doc = Document()
    doc.add_heading(f'arXiv Results for: {query} (Sorted by Newest)', 0)

    for i, entry in enumerate(entries, 1):
        doc.add_heading(f'{i}. {entry.title}', level=1)
        authors = ', '.join(author.name for author in entry.authors)
        doc.add_paragraph(f"Authors: {authors}")
        doc.add_paragraph(f"Published: {entry.published}")
        doc.add_paragraph("Abstract:", style='Intense Quote')
        doc.add_paragraph(entry.summary.strip())
        doc.add_paragraph(f"PDF Link: {entry.link}")
        doc.add_paragraph("-" * 50)

    doc.save(filename)
    print(f"Saved {len(entries)} results to '{filename}', sorted by newest first.")

# Example usage
search_arxiv_to_word_sorted("Blind channel estimation", 100)


✅ Saved 100 results to 'Blind channel estimation.docx', sorted by newest first.
